In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/layer10/valid.csv
/kaggle/input/layer10/train.csv
/kaggle/input/layer10/test.csv


In [5]:
import pandas as pd
import numpy as np
from pandas import Series

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [6]:
train_df = pd.read_csv('/kaggle/input/layer10/train.csv')
valid_df = pd.read_csv('/kaggle/input/layer10/valid.csv')
test_df = pd.read_csv('/kaggle/input/layer10/test.csv')

In [7]:
train_df.shape

(28520, 772)

In [8]:
train_df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,-0.027083,0.072947,-0.093659,0.053418,-0.085516,-0.102610,-0.021217,0.016162,-0.184269,0.110335,...,-0.183643,0.091299,-0.037097,0.042607,-0.034361,-0.013748,45,NaN,1,6
1,0.070195,0.228641,-0.132860,-0.077761,-0.054993,-0.210365,0.127747,-0.132385,-0.161366,0.172764,...,-0.123668,0.029626,-0.027345,0.055223,-0.179725,0.136841,45,NaN,1,6
2,0.164312,0.052808,-0.058510,0.104724,-0.025886,-0.101427,-0.047177,0.091298,-0.094569,0.088062,...,0.075410,0.070125,0.043022,0.012972,-0.028920,0.096725,45,NaN,1,6
3,0.029730,0.113737,0.061113,-0.099329,-0.111600,-0.245942,0.086520,0.071996,0.028319,0.207910,...,-0.062511,-0.226912,-0.046011,0.011282,-0.095167,0.039979,45,NaN,1,6
4,0.031364,0.142409,-0.160743,-0.076594,-0.062412,-0.264732,0.079197,0.026060,-0.217023,0.084656,...,-0.193882,0.107297,-0.042355,0.046763,-0.192469,0.006463,45,NaN,1,6


In [9]:
missing_columns = train_df.columns[train_df.isnull().any()]
missing_counts = train_df[missing_columns].isnull().sum()

print('Missing Columns and Counts')
for column in missing_columns:
    print( str(column) +' : '+ str(missing_counts[column]))

Missing Columns and Counts
label_2 : 480


In [10]:
train_data = train_df.copy()
valid_data = valid_df.copy()
test_data = test_df.copy()

In [11]:
train_df.describe()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
count,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,...,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28040.000000,28520.000000,28520.000000
mean,0.035587,0.078413,-0.040854,0.079167,-0.033728,-0.129528,-0.004608,-0.016071,-0.080182,0.047877,...,-0.121131,0.024537,-0.023062,0.053033,-0.060110,-0.031766,30.498843,27.975107,0.799299,5.997125
std,0.066663,0.051860,0.056119,0.061433,0.055724,0.078193,0.055673,0.067243,0.070111,0.061358,...,0.082048,0.065089,0.063021,0.081957,0.061217,0.074294,17.328389,5.735913,0.400532,2.375567
min,-0.278307,-0.108783,-0.374459,-0.230660,-0.382474,-0.464895,-0.218843,-0.385482,-0.386470,-0.396842,...,-0.400943,-0.261302,-0.271254,-0.383916,-0.350115,-0.296535,1.000000,22.000000,0.000000,0.000000
25%,-0.003948,0.044925,-0.074197,0.040072,-0.069260,-0.182223,-0.041694,-0.057925,-0.123943,0.008245,...,-0.172992,-0.015233,-0.067517,-0.006157,-0.099727,-0.081620,15.000000,25.000000,1.000000,6.000000
50%,0.032880,0.076560,-0.039102,0.079309,-0.031002,-0.124762,-0.006371,-0.015115,-0.077953,0.046913,...,-0.129561,0.023859,-0.027330,0.050707,-0.056456,-0.031862,30.000000,27.000000,1.000000,6.000000
75%,0.068754,0.109380,-0.005096,0.118440,0.004372,-0.072187,0.030630,0.025157,-0.036984,0.086407,...,-0.082763,0.062431,0.016195,0.110795,-0.019083,0.017753,46.000000,30.000000,1.000000,6.000000
max,0.537892,0.365866,0.329586,0.397721,0.214701,0.215949,0.461929,0.373033,0.433161,0.333654,...,0.345534,0.487504,0.596317,0.348570,0.334053,0.395296,60.000000,61.000000,1.000000,13.000000


In [12]:
from sklearn.preprocessing import RobustScaler # eliminate outliers

x_train = {}
x_valid = {}
x_test = {}

y_train = {}
y_valid = {}
y_test = {}

#create dictionaries for each label
for target_label in ['label_1','label_2','label_3','label_4']:

  if target_label == "label_2":
    train = train_df[train_df['label_2'].notna()]
    valid = valid_df[valid_df['label_2'].notna()]
  else:
    train = train_df
    valid = valid_df

  test = test_df

  scaler = RobustScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(train.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_train[target_label] = train[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(valid.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_valid  [target_label] = valid[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(test.drop(["ID"],axis=1)), columns=[f'feature_{i}' for i in range(1,769)])

In [13]:
x_train_df = x_train['label_2'].copy()
y_train_df = y_train['label_2'].copy()

x_valid_df = x_valid['label_2'].copy()
y_valid_df = y_valid['label_2'].copy()

x_test_df = x_test['label_2'].copy()

# Cross Validation

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold

# Perform cross-validation
scores = cross_val_score(SVC(), x_train_df, y_train_df, cv=5, scoring='accuracy')

mean_accuracy = scores.mean()
std_accuracy = scores.std()
# Print the cross-validation scores
print('Support Vector Machines')
print('\n')
print("Cross-validation scores:", scores)
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")

Support Vector Machines


Cross-validation scores: [0.42350214 0.64497147 0.70845221 0.67510699 0.5064194 ]
Mean Accuracy: 0.59
Standard Deviation: 0.11


# Feature Engineering

In [15]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.975, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df))
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df))
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df))
print('Shape after PCA: ',x_train_df_pca.shape)

Shape after PCA:  (28040, 387)


In [16]:
from sklearn import metrics

# SVM

In [17]:
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

classifier = svm.SVC(kernel='linear', C=1)

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

acc_score:  0.8260869565217391


In [18]:
# from sklearn.svm import SVC
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform, randint
# import numpy as np

# param_dist = {
#     'C': [100,10,1,0,0.1,0.01],
#     'kernel': ['rbf','linear','poly','sigmoid'],
#     'gamma': ['scale','auto'],
#     'degree': [1,2,3,4],
#     'class_weight' : ['none','balanced']
# }

# svm = SVC()

# random_search = RandomizedSearchCV(
#     svm, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42, scoring='accuracy'
# )

# random_search.fit(x_train_df_pca, y_train_df)

# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

# print("best parameters:", best_params)

Label 1 
best parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'class_weight': 'balanced', 'C': 100}


In [19]:
from sklearn import svm

classifier = svm.SVC(kernel='rbf', C=100, gamma='scale', degree=4, class_weight='balanced')

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_predict_after_pca = classifier.predict(x_test_df_pca)



acc_score:  0.9510869565217391


# RandomForest

In [20]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42)

classifier.fit(x_train_df, y_train_df)

y_valid_pred = classifier.predict(x_valid_df)

print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_pred = classifier.predict(x_test_df)

accuracy_score:  0.779891304347826


# CSV Creation

In [21]:
output_df=pd.DataFrame(columns=["ID","label_1","label_2","label_3","label_4"])

In [22]:
IDs = list(i for i in range(1, len(test_df)+1))
output_df["ID"] = IDs

In [23]:
output_df["label_2"] = y_test_predict_after_pca

In [24]:
output_df

,ID,label_1,label_2,label_3,label_4
0,1,NaN,22.0,NaN,NaN
1,2,NaN,25.0,NaN,NaN
2,3,NaN,30.0,NaN,NaN
3,4,NaN,27.0,NaN,NaN
4,5,NaN,29.0,NaN,NaN
...,...,...,...,...,...
739,740,NaN,26.0,NaN,NaN
740,741,NaN,24.0,NaN,NaN
741,742,NaN,27.0,NaN,NaN
742,743,NaN,32.0,NaN,NaN


In [25]:
output_df.to_csv('/kaggle/working/output10_2.csv',index=False)